In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import pandas as pd
import torch
import numpy as np

In [2]:
df = pd.read_excel("/project/class/bii_sdad_dspg/uva_2021/binn/data.xlsx") #This excel file was pre processed to have columns CompanyName and CompanyName2
df_yes = df[df.YN_INNOVATION == 'yes']
df_yes = df_yes[["full_text","CompanyName","CompanyName2"]]
df_yes

,full_text,CompanyName,CompanyName2
0,Creaform and Metrologic Group Announce Worldwi...,Creaform,Metrologic Group
7,BoardwalkTech; Boardwalktech Announces BW.Supp...,BoardwalkTech,NaN
12,"American LegalNet, Inc. American LegalNet Anno...","American LegalNet, Inc.",American LegalNet (ALN)
13,Barron's Blog: HP's Donatelli: Pulling Stuff O...,HP,Hewlett-Packard ( HPQ)
16,eASIC Corporation; eASIC Joins the OpenPOWER F...,eASIC Corporation,eASIC
...,...,...,...
570,Application Software Companies; Red Hat Breaks...,Red Hat,NaN
584,Rocket Fuel Introduces Next Generation Mobile ...,Rocket Fuel,NaN
588,Shopify Develops New Program for Expert Users ...,Shopify,NaN
596,Transaction Network Services; TNS Unveils Uniq...,Transaction Network Services,NaN


In [3]:
df_new = pd.read_csv("/home/asr8r/git/dspg21binn/data/DNA_Food_Articles_2017_no_rep.csv", index_col=0) ## Reading in data
df_new_yn = pd.read_csv("/home/asr8r/git/dspg21binn/data/100k_articles_index_innovation_classification_prediction_diggy.csv", index_col=0) ## Reading in data
df_new = df_new.merge(df_new_yn, on = "an")
df_new = df_new[df_new.pred_y == 1]
df_new = df_new.reset_index(inplace=False, drop = True)

In [4]:
def ner(df_new):
    df_new.ner_org = np.NaN
    df_new.ner_score = np.NaN

    device = torch.device("cuda")
    tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
    model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
    nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
    for article in range(len(df_new)):
        print("Row number:", article)
        example = df_new.loc[article,'full_text']     #full_text of article
        ner_results = nlp(example)     #running nlp 
        ner_df = pd.DataFrame(ner_results)     #results of nlp
        if not ner_df.empty: 
            Org = ner_df[ner_df.entity_group == ("ORG") ] #all the ORG's in results 
            Org = Org.sort_values(by=['score'],ascending = False)
            Org.drop_duplicates('word', inplace = True)
            ner_org = Org.head(3).word.to_list() ## Taking top 3 companies from each row
            ner_score = Org.head(3).score.to_list()
        else:
            ner_org = None
            ner_score = None
        
        df_new.loc[article, "ner_org"] = str(ner_org)
        df_new.loc[article, "ner_score"] = str(ner_score)

In [5]:
ner(df_new)

Row number: 0
Row number: 1
Row number: 2
Row number: 3
Row number: 4
Row number: 5
Row number: 6
Row number: 7
Row number: 8
Row number: 9
Row number: 10
Row number: 11
Row number: 12
Row number: 13
Row number: 14
Row number: 15
Row number: 16
Row number: 17
Row number: 18
Row number: 19
Row number: 20
Row number: 21
Row number: 22
Row number: 23
Row number: 24
Row number: 25
Row number: 26
Row number: 27
Row number: 28
Row number: 29
Row number: 30
Row number: 31
Row number: 32
Row number: 33
Row number: 34
Row number: 35
Row number: 36
Row number: 37
Row number: 38
Row number: 39
Row number: 40
Row number: 41
Row number: 42
Row number: 43
Row number: 44
Row number: 45
Row number: 46
Row number: 47
Row number: 48
Row number: 49
Row number: 50
Row number: 51
Row number: 52
Row number: 53
Row number: 54
Row number: 55
Row number: 56
Row number: 57
Row number: 58
Row number: 59
Row number: 60
Row number: 61
Row number: 62
Row number: 63
Row number: 64
Row number: 65
Row number: 66
Row n

In [6]:
df_new["ner_org"] = df_new["ner_org"].str.strip("[") ## Commas at the start
df_new["ner_org"] = df_new["ner_org"].str.strip("]") ## Commas at the start
df_new["ner_org"] = df_new["ner_org"].str.strip("'") ## Commas at the start
df_new["ner_org"] = df_new["ner_org"].str.split("', '",expand=False) ## List
df_new["ner_org"] = df_new["ner_org"].replace(np.NaN, "") ## Getting rid of floats to convert to sets

for i in range(len(df_new)):
    df_new["ner_org"].loc[i] = list(set(df_new["ner_org"].loc[i])) ## Unique companies list



max_companies = 0
for row in df_new["ner_org"]:
    max = len(row)
    if max > max_companies:
        max_companies = max
        
list_of_column_labels = []

for i in ran

ge(max_companies):
    list_of_column_labels.append("company_name_"+str(i+1))
        
df_new = df_new.join(pd.DataFrame(df_new["ner_org"].to_list(), columns = list_of_column_labels))

/home/far3jh/.conda/envs/DiggyConda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
df_new

,index,copyright,subject_codes,art,modification_datetime,body,company_codes_occur_ticker_exchange,company_codes_occur,company_codes_about,company_codes_lineage,...,date,year,full_text,pred_y,pred_prob,ner_org,ner_score,company_name_1,company_name_2,company_name_3
0,18,© Copyright 2017 Computer Weekly News via via ...,",ccat,",NaN,1487890959000,The Access Valve for ThingWorx integrates an o...,NaN,NaN,NaN,NaN,...,2017-02-23 00:00:00.000,2017,"Certified Security Solutions, Inc. CSS Release...",1,0.988424,"[Computer Weekly News, Certified Security Solu...","[0.9988489866256713, 0.9974478284517924, 0.988...",Computer Weekly News,"Certified Security Solutions, Inc",ThingWorx
1,26,Copyright 2017 Telecompaper. All Rights Reserv...,",ccat,c22,cprdop,c24,ncat,nfact,nfcpin,",NaN,1508513602000,Cortana on the new speaker will look up inform...,NaN,",mcrost,harmki,gognew,bstbuy,amzcom,",",mcrost,harint,bstbuy,harmki,",",sansel,",...,2017-10-20 00:00:00.000,2017,Harman Kardon launches smart speaker with Micr...,1,0.974480,"[Microsoft, Amazon, Google]","[0.9988275766372681, 0.9987722039222717, 0.997...",Microsoft,Amazon,Google
2,36,"© Copyright 2017 GlobeNewswire, Inc. All Right...",",cpartn,ccpint,npress,c11,ccat,ncat,",NaN,1484229727000,"According to Lee Nagel, vice president of mark...",NaN,",orncmi,onlnfr,linkd,",NaN,NaN,...,2017-01-12 14:00:04.057,2017,Izenda Partners with Orion Communications to P...,1,0.913724,"[Orion Communications, Orion, Orion Communicat...","[0.9992634057998657, 0.999161958694458, 0.9967...",Orion Communications,Orion,"Orion Communications, Inc"
3,57,© Copyright 2017 Computer Weekly News via via ...,",ccat,",NaN,1503007605000,Given its aggressive plans to take more market...,NaN,",hubspo,",",hubspo,",NaN,...,2017-08-17 00:00:00.000,2017,Qstream; HubSpot Signs Global Agreement with Q...,1,0.778763,"[Computer Weekly News, HubSpot, Hu]","[0.9976779818534851, 0.9966532588005066, 0.996...",Computer Weekly News,HubSpot,Hu
4,142,"© 2017, M2 Communications. All rights reserved.",",cpartn,ccexpi,ccrm,npress,c31,ccat,ncat,nfact...",NaN,1499778788000,Developed in partnership with the Infor Custom...,NaN,",agilis,",",agilis,",NaN,...,2017-07-11 00:00:00.000,2017,Infor Amplifies Customer Service by Delivering...,1,0.950500,"[Y, Con, In]","[0.9451677799224854, 0.8110693693161011, 0.764...",Y,Con,In
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21911,1292888,Copyright © 2017 The Denver Post Corp. Distrib...,",gcat,",NaN,1508395845000,"Dubbed the Fall Creators Update, the package w...",NaN,",sansel,mcrost,hewlp,acrser,",",mcrost,",NaN,...,2017-10-18 00:00:00.000,2017,Microsoft's Windows 10 update bets on a bigger...,1,0.995043,"[HP, Microsoft, Samsung]","[0.9986824989318848, 0.9984120726585388, 0.997...",HP,Microsoft,Samsung
21912,1292910,"(c) Copyright 2017, The Des Moines Register. A...",",ccat,",Amazon,1488359396000,"For example, a business might store its video ...",NaN,",zxamaz,gdsicz,bzzfd,",",amzcom,zxamaz,",NaN,...,2017-03-01 00:00:00.000,2017,Server fail wreaks havoc on websites; Amazon W...,1,0.628089,"[Amazon Web Services, Spotify, Amazon]","[0.9991624355316162, 0.9973240494728088, 0.995...",Amazon Web Services,Spotify,Amazon
21913,1292940,"Copyright 2017, The Seattle Times Company. All...",",ghack,gcat,gcrim,ncat,nfact,nfcpex,",Patrick Marshall P-Tech columnist Patrick Mars...,1510391547000,A: I do believe in “innocent until proven guil...,NaN,",mcrost,gognew,comcst,",",mcrost,",NaN,...,2017-11-11 00:00:00.000,2017,Run security software that you feel good about...,1,0.606872,"[CenturyLink, Comcast, Department of Homeland ...","[0.9992834478616714, 0.9989486535390218, 0.998...",CenturyLink,Comcast,Department of Homeland Security
21914,1292982,Copyright 2017. NewsCheckMedia LLC. All righ...,",c315,cpartn,ccat,",NaN,1492963215000,"""We will work closely together on cloud-based ...",NaN,",wynrel,mcrost,dlby,",",mcrost,",NaN,...,2017-04-23 00:00:00.000,2017,Avid Moves It All To The

In [10]:
df_new.to_csv('/project/class/bii_sdad_dspg/uva_2021/binn/bert-base-NER_100k_articles.csv', index = False)

In [ ]:
#df_new.company_codes = df_new.company_codes.str.strip(",") ## Commas at the start
#df_new.company_codes = df_new.company_codes.str.split(",",expand=False) ## List
#df_new.company_codes = df_new.company_codes.replace(np.NaN, "") ## Getting rid of floats to convert to sets

#for i in range(len(df_new)):
#    df_new.company_codes.loc[i] = list(set(df_new.company_codes.loc[i])) ## Unique companies list

#max_companies = 0
#for row in df_new.company_codes:
#    max = len(row)
#    if max > max_companies:
#        max_companies = max
        
#list_of_column_labels = []

#for i in range(max_companies):
#    list_of_column_labels.append("company_name_"+str(i+1))
    
#df_new = df_new.join(pd.DataFrame(df_new.company_codes.to_list(), columns = list_of_column_labels))

In [ ]:
## END